# Load stage-0.csv

In [1]:
import pandas as pd
df = pd.read_csv('../datasets/stage-0.csv')
df

,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths,Active,Incidence_Rate,Case-Fatality_Ratio
0,-,Hubei,Mainland China,1/28/20 23:00,NaN,NaN,3554.0,80.0,125.0,NaN,NaN,NaN
1,-,Guangdong,Mainland China,1/28/20 23:00,NaN,NaN,207.0,4.0,NaN,NaN,NaN,NaN
2,-,Zhejiang,Mainland China,1/28/20 23:00,NaN,NaN,173.0,3.0,NaN,NaN,NaN,NaN
3,-,Henan,Mainland China,1/28/20 23:00,NaN,NaN,168.0,NaN,1.0,NaN,NaN,NaN
4,-,Hunan,Mainland China,1/28/20 23:00,NaN,NaN,143.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
618545,-,Unknown,Colombia,2020-07-17 22:34:48,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
618546,Dillingham,Alaska,US,2020-07-17 18:35:12,59.796037,-158.238194,1.0,0.0,0.0,1.0,20.341741,0.0
618547,-,Grand Princess,Canada,2020-07-13 12:34:33,NaN,NaN,13.0,13.0,0.0,0.0,NaN,0.0
618548,Lewis,Idaho,US,2020-07-10 02:34:22,46.233153,-116.434146,1.0,0.0,0.0,1.0,26.055237,0.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618550 entries, 0 to 618549
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Admin2               618550 non-null  object 
 1   Province_State       618550 non-null  object 
 2   Country_Region       618550 non-null  object 
 3   Last_Update          618550 non-null  object 
 4   Lat                  604196 non-null  float64
 5   Long_                604196 non-null  float64
 6   Confirmed            618522 non-null  float64
 7   Recovered            618125 non-null  float64
 8   Deaths               618072 non-null  float64
 9   Active               610696 non-null  float64
 10  Incidence_Rate       388203 non-null  float64
 11  Case-Fatality_Ratio  390637 non-null  float64
dtypes: float64(8), object(4)
memory usage: 56.6+ MB


# Step-7

### Regex Filter

#### Pre-Step

In [3]:
# Buat 1 kolom baru dengan nilai default 'N'.
df['lu_temp'] = 'N'

#### Gradual step-0: Snip terlebih dahulu semua format yang di dapat pertama kali

In [4]:
%%script false --no-raise-error
df[df['lu_temp']=='N']['Last_Update'].head()

#### Function

In [5]:
# Hitung banyak tanggal dengan format tertentu
# Jika ditemukan tanggal dengan format seperti X, maka nilai 'N' pada 'lu_temp' diubah ke 'Y' 
import re

def pattern_checker(data, pattern):
    for i in range(len(data)):
        get_loc_lastUpdate = data.columns.get_loc('Last_Update')
        date_txt = data.iat[i, get_loc_lastUpdate]
        result = re.match(pattern, date_txt)
        if result != None:
            get_loc_lu_temp = data.columns.get_loc('lu_temp')
            data.iat[i, get_loc_lu_temp] = 'Y'

def pattern_checker_report(data, pattern):
    pattern_checker(data, pattern)
    print([len(data[data['lu_temp']=='Y']), len(data) == len(data[data['lu_temp']=='Y'])])

def convert_a(x):
    x = x.split(' ')
    x[0] = x[0].split('/')
    if int(x[0][0]) < 10:
        x[0][0] = '0'+x[0][0]
    else:
        pass
    if int(x[0][1]) < 10:
        x[0][1] = '0'+x[0][1]
    else:
        pass
    if int(x[0][2]) < 2000:
        x[0][2] = '20'+x[0][2]
    else:
        pass
    x[0][0], x[0][2] = x[0][2], x[0][0]
    x[0][1], x[0][2] = x[0][2], x[0][1]
    x[0] = '-'.join(x[0])
    x[1] = x[1].split(':')
    if int(x[1][0]) < 10:
        x[1][0] = '0'+x[0][1]
    else:
        pass
    x[1] = ':'.join(x[1])
    x[1] += ':00'
    return ' '.join(x)

def convert_c(x):
    return ' '.join(x.split('T'))

def higher_order_converter(df, pattern, convert_func):
    df['lu_temp'] = 'N'
    pattern_checker(df, pattern)
    x = df[df['lu_temp']=='Y'].index
    get_loc_lu = df.columns.get_loc('Last_Update')
    for i in x:
        df.iat[i, get_loc_lu] = convert_func(df.iat[i, get_loc_lu])

#### Gradual step-1 : Tulis secara gradual semua ekspresi Regex yang ditemukan pada Gradual step-2 

In [6]:
# 1/28/20 23:00 (Pattern A)
a = '^([1-9]|1[0-2])/([1-9]|[12][0-9]|3[01])/20 (00|[0-9]|1[0-9]|2[0-3]):([0-9]|[0-5][0-9])$'
# 2020-07-17 22:34:48 (Pattern B) ---> Ini adalah pattern yang kita inginkan
b = '^202[0-3]-(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01]) (0[0-9]|1[0-9]|2[0-3]):([0-5][0-9]):([0-5][0-9])$'
# 2020-03-06T14:23:04 (Pattern C)
c = '^202[0-3]-(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])T(0[0-9]|1[0-9]|2[0-3]):([0-5][0-9]):([0-5][0-9])$'
# 1/22/2020 17:00 (Pattern D)
d = '^[1-9]/([1-9]|[12][0-9]|3[01])/20[0-9]{2} (00|[0-9]|1[0-9]|2[0-3]):([0-9]|[0-5][0-9])$'
pattern = [a, b, c, d]

#### Gradual step-2 : Regex Filter. Dan kemudian kembali ke Gradual step-0

In [7]:
%%script false --no-raise-error

# Buat 1 kolom baru dengan nilai default 'N'.
df['lu_temp'] = 'N'

for i in range(len(pattern)):
    pattern_checker_report(df, pattern[i])

print(len(df[df['lu_temp']=='N'])) # memastikan bahwa sudah tidak ada lagi value 'N' pada kolom 'lu_temp'

#### Gradual step, Finish : Aktifkan script-false pada Gradual step-0 dan Gradual step-2

### Converter

In [8]:
df.drop('lu_temp', axis=1, inplace=True) # drop kolom 'lu_temp'
higher_order_converter(df, pattern[0], convert_a)
higher_order_converter(df, pattern[2], convert_c)
higher_order_converter(df, pattern[3], convert_a)

In [9]:
# %%script false --no-raise-error

# Buat 1 kolom baru dengan nilai default 'N'.
df['lu_temp'] = 'N'

for i in range(len(pattern)):
    pattern_checker_report(df, pattern[i])

print(len(df[df['lu_temp']=='N'])) # memastikan bahwa sudah tidak ada lagi value 'N' pada kolom 'lu_temp'

df.drop('lu_temp', axis=1, inplace=True) # drop kolom 'lu_temp'

df['Last_Update'] = pd.to_datetime(df['Last_Update'])

[0, False]
[618550, True]
[618550, True]
[618550, True]
0


In [10]:
import os
os.chdir('..')
os.chdir('datasets')
df.to_csv("stage-1.csv", index=False)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618550 entries, 0 to 618549
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Admin2               618550 non-null  object        
 1   Province_State       618550 non-null  object        
 2   Country_Region       618550 non-null  object        
 3   Last_Update          618550 non-null  datetime64[ns]
 4   Lat                  604196 non-null  float64       
 5   Long_                604196 non-null  float64       
 6   Confirmed            618522 non-null  float64       
 7   Recovered            618125 non-null  float64       
 8   Deaths               618072 non-null  float64       
 9   Active               610696 non-null  float64       
 10  Incidence_Rate       388203 non-null  float64       
 11  Case-Fatality_Ratio  390637 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 56.6+ MB
